<a href="https://colab.research.google.com/github/alanssitis/ROS-Intro-Project/blob/main/MobileBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch] datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.1 MB/s eta 0:00:00


In [2]:
from typing import List, Optional, Tuple, Union

import evaluate
import numpy as np
import torch

from torch import nn
from datasets import load_dataset
from transformers import (
    MobileBertModel,
    MobileBertForSequenceClassification,
    MobileBertTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Test The Corpus of Linguistic Acceptability

In [3]:
tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
raw_datasets = load_dataset("glue", "cola",)
def tokenize_raw_datasets(raw_datasets):
    return tokenizer(raw_datasets["sentence"], return_tensors="pt", max_length=128, padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_raw_datasets, batched=True)

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [5]:
metric = evaluate.load("matthews_correlation")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.497400,0.485400,0.508885
2,0.354700,0.514453,0.544301
3,0.238200,0.799014,0.536891


TrainOutput(global_step=3207, training_loss=6206.4469615387925, metrics={'train_runtime': 567.9131, 'train_samples_per_second': 45.171, 'train_steps_per_second': 5.647, 'total_flos': 402165422046720.0, 'train_loss': 6206.4469615387925, 'epoch': 3.0})

## Test The Stanford Sentiment Treebank

In [7]:
tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
raw_datasets = load_dataset("glue", "sst2",)
def tokenize_raw_datasets(raw_datasets):
    return tokenizer(raw_datasets["sentence"], return_tensors="pt", max_length=128, padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_raw_datasets, batched=True)


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.235800,0.393859,0.810913
2,0.179500,0.410118,0.821139
3,0.110600,0.439626,0.817160


TrainOutput(global_step=25257, training_loss=412.0922419551848, metrics={'train_runtime': 4371.8908, 'train_samples_per_second': 46.215, 'train_steps_per_second': 5.777, 'total_flos': 3167517133601280.0, 'train_loss': 412.0922419551848, 'epoch': 3.0})

## Test Microsoft Research Paraphrase Corpus

In [10]:
tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
raw_datasets = load_dataset("glue", "mrpc",)
def tokenize_raw_datasets(raw_datasets):
    return tokenizer(raw_datasets["sentence1"], raw_datasets["sentence2"], return_tensors="pt", max_length=128, padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_raw_datasets, batched=True)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [12]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.349117,0.850490
2,28246.074000,0.460115,0.818627
3,0.344900,0.477430,0.877451


TrainOutput(global_step=1377, training_loss=10256.57392471635, metrics={'train_runtime': 245.0398, 'train_samples_per_second': 44.907, 'train_steps_per_second': 5.619, 'total_flos': 172511141160960.0, 'train_loss': 10256.57392471635, 'epoch': 3.0})

## Test Semantic Textual Similarity Benchmark

In [18]:
tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=1)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
raw_datasets = load_dataset("glue", "stsb",)
def tokenize_raw_datasets(raw_datasets):
    return tokenizer(raw_datasets["sentence1"], raw_datasets["sentence2"], return_tensors="pt", max_length=128, padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_raw_datasets, batched=True)

In [16]:
metric = evaluate.load("pearsonr")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Pearsonr
1,866697154134.015991,0.685251,nan
2,0.577600,0.576279,nan
3,0.303300,0.488535,nan


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


TrainOutput(global_step=2157, training_loss=200903373698.5261, metrics={'train_runtime': 397.0762, 'train_samples_per_second': 43.435, 'train_steps_per_second': 5.432, 'total_flos': 270376670207232.0, 'train_loss': 200903373698.5261, 'epoch': 3.0})

## Test Quora Question Pairs

In [119]:
tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [120]:
raw_datasets = load_dataset("glue", "qqp",)
def tokenize_raw_datasets(raw_datasets):
    return tokenizer(raw_datasets["question1"], raw_datasets["question2"], return_tensors="pt", padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_raw_datasets, batched=True)

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [121]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [122]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(5000)),
    eval_dataset=tokenized_datasets["validation"].select(range(1000)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,13413.028000,0.400634,0.807000
2,0.402800,0.452888,0.809000
3,0.701000,0.612111,0.815000


TrainOutput(global_step=1875, training_loss=3577.150262141927, metrics={'train_runtime': 379.2943, 'train_samples_per_second': 39.547, 'train_steps_per_second': 4.943, 'total_flos': 273549265737120.0, 'train_loss': 3577.150262141927, 'epoch': 3.0})

## Test MultiNLI

In [106]:
tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=3)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [113]:
train_datasets = load_dataset("glue", "mnli", split="train").select(range(9000))
eval_matched_datasets, eval_mismatched_datasets = load_dataset("glue", "mnli", split=["validation_matched", "validation_mismatched"])

def tokenize_raw_datasets(raw_datasets):
    return tokenizer(raw_datasets["premise"], raw_datasets["hypothesis"], return_tensors="pt", padding=True, truncation=True)

tokenized_train = train_datasets.map(tokenize_raw_datasets, batched=True)
tokenized_eval_matched = eval_matched_datasets.map(tokenize_raw_datasets, batched=True)
tokenized_eval_mismatched = eval_mismatched_datasets.map(tokenize_raw_datasets, batched=True)


Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [114]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [118]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval_matched,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval_mismatched,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.733200,0.717606,0.705553
2,0.531100,0.645390,0.745899
3,0.339200,1.069469,0.749669


Epoch,Training Loss,Validation Loss,Accuracy
1,0.298200,27.219135,0.743389
2,0.202500,4.966821,0.742880
3,0.117700,2.726878,0.747254


TrainOutput(global_step=3375, training_loss=0.3302736386899595, metrics={'train_runtime': 840.9044, 'train_samples_per_second': 32.108, 'train_steps_per_second': 4.014, 'total_flos': 772760846525184.0, 'train_loss': 0.3302736386899595, 'epoch': 3.0})

## Test Question NLI

In [88]:
tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [92]:
train_datasets = load_dataset("glue", "qnli", split="train").select(range(5000))
eval_datasets = load_dataset("glue", "qnli", split="validation").select(range(5000))

def tokenize_raw_datasets(raw_datasets):
    return tokenizer(raw_datasets["question"], raw_datasets["sentence"], return_tensors="pt", max_length=128, padding="max_length", truncation=True)

tokenized_train_datasets = train_datasets.map(tokenize_raw_datasets, batched=True)
tokenized_eval_datasets = eval_datasets.map(tokenize_raw_datasets, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [94]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [95]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_eval_datasets,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,39070.668000,0.388727,0.839200
2,3.351600,0.415466,0.835000
3,0.317300,0.620195,0.844600


TrainOutput(global_step=1875, training_loss=10419.874475976563, metrics={'train_runtime': 452.3475, 'train_samples_per_second': 33.16, 'train_steps_per_second': 4.145, 'total_flos': 235156953600000.0, 'train_loss': 10419.874475976563, 'epoch': 3.0})

## Test Recognizing Textual Entailment

In [96]:
tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [98]:
raw_datasets = load_dataset("glue", "rte",)
def tokenize_raw_datasets(raw_datasets):
    return tokenizer(raw_datasets["sentence1"], raw_datasets["sentence2"], return_tensors="pt", padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_raw_datasets, batched=True)

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [99]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [100]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.657082,0.602888
2,29089.734000,0.601782,0.649819
3,29089.734000,0.662739,0.671480


TrainOutput(global_step=936, training_loss=15539.621208451752, metrics={'train_runtime': 188.4762, 'train_samples_per_second': 39.634, 'train_steps_per_second': 4.966, 'total_flos': 263835078957000.0, 'train_loss': 15539.621208451752, 'epoch': 3.0})

## Test Winograd NLI

In [101]:
tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [102]:
raw_datasets = load_dataset("glue", "wnli",)
def tokenize_raw_datasets(raw_datasets):
    return tokenizer(raw_datasets["sentence1"], raw_datasets["sentence2"], return_tensors="pt", padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_raw_datasets, batched=True)

Generating train split:   0%|          | 0/635 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/71 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/146 [00:00<?, ? examples/s]

Map:   0%|          | 0/635 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/71 [00:00<?, ? examples/s]

Map:   0%|          | 0/146 [00:00<?, ? examples/s]

In [103]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [104]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.894971,0.436620
2,No log,0.759654,0.563380
3,No log,0.735726,0.323944


TrainOutput(global_step=240, training_loss=42939.666666666664, metrics={'train_runtime': 46.3611, 'train_samples_per_second': 41.091, 'train_steps_per_second': 5.177, 'total_flos': 25198537309200.0, 'train_loss': 42939.666666666664, 'epoch': 3.0})